In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact, FloatSlider, Checkbox
import ipywidgets as widgets


In [ ]:

# === 3D Cube Definition in Homogeneous Coordinates ===
cube = np.array([
    [0, 0, 0, 1],
    [1, 0, 0, 1],
    [1, 1, 0, 1],
    [0, 1, 0, 1],
    [0, 0, 1, 1],
    [1, 0, 1, 1],
    [1, 1, 1, 1],
    [0, 1, 1, 1]
]).T  # shape (4, 8)

edges = [
    (0, 1), (1, 2), (2, 3), (3, 0),  # bottom
    (4, 5), (5, 6), (6, 7), (7, 4),  # top
    (0, 4), (1, 5), (2, 6), (3, 7)   # sides
]


In [ ]:
def apply_transform(matrix, shape):
    return matrix @ shape

def plot_3d(transformed, title):
    fig = plt.figure(figsize=(6, 6))
    ax = fig.add_subplot(111, projection='3d')
    
    # Original cube (light gray)
    for i, j in edges:
        ax.plot([cube[0, i], cube[0, j]],
                [cube[1, i], cube[1, j]],
                [cube[2, i], cube[2, j]],
                'gray', linestyle='--', linewidth=1)

    # Transformed cube (red)
    for i, j in edges:
        ax.plot([transformed[0, i], transformed[0, j]],
                [transformed[1, i], transformed[1, j]],
                [transformed[2, i], transformed[2, j]],
                'r', linewidth=2)

    ax.set_title(title)
    ax.set_xlim(-2, 2)
    ax.set_ylim(-2, 2)
    ax.set_zlim(-2, 2)
    ax.set_box_aspect([1,1,1])
    plt.show()


In [ ]:
# === SCALING ===
def scale_plot(sx=1.0, sy=1.0, sz=1.0):
    S = np.array([
        [sx, 0,  0,  0],
        [0,  sy, 0,  0],
        [0,  0,  sz, 0],
        [0,  0,  0,  1]
    ])
    T = apply_transform(S, cube)
    plot_3d(T, f"Scaling: sx={sx}, sy={sy}, sz={sz}")

interact(scale_plot,
         sx=FloatSlider(value=1, min=-2, max=2, step=0.1),
         sy=FloatSlider(value=1, min=-2, max=2, step=0.1),
         sz=FloatSlider(value=1, min=-2, max=2, step=0.1))


In [ ]:

# === SHEARING ===
def shear_plot(shxy=0.0, shxz=0.0, shyx=0.0, shyz=0.0, shzx=0.0, shzy=0.0):
    Sh = np.array([
        [1,   shxy, shxz, 0],
        [shyx, 1,   shyz, 0],
        [shzx, shzy, 1,   0],
        [0,    0,    0,   1]
    ])
    T = apply_transform(Sh, cube)
    plot_3d(T, f"Shearing")

interact(shear_plot,
         shxy=FloatSlider(value=0, min=-1, max=1, step=0.1),
         shxz=FloatSlider(value=0, min=-1, max=1, step=0.1),
         shyx=FloatSlider(value=0, min=-1, max=1, step=0.1),
         shyz=FloatSlider(value=0, min=-1, max=1, step=0.1),
         shzx=FloatSlider(value=0, min=-1, max=1, step=0.1),
         shzy=FloatSlider(value=0, min=-1, max=1, step=0.1))


In [ ]:

# === ROTATION ===
def rotate_plot(rx=0, ry=0, rz=0):
    # Convert to radians
    rx, ry, rz = np.radians(rx), np.radians(ry), np.radians(rz)
    
    Rx = np.array([
        [1, 0,           0,          0],
        [0, np.cos(rx), -np.sin(rx), 0],
        [0, np.sin(rx),  np.cos(rx), 0],
        [0, 0,           0,          1]
    ])
    
    Ry = np.array([
        [ np.cos(ry), 0, np.sin(ry), 0],
        [ 0,          1, 0,          0],
        [-np.sin(ry), 0, np.cos(ry), 0],
        [ 0,          0, 0,          1]
    ])
    
    Rz = np.array([
        [np.cos(rz), -np.sin(rz), 0, 0],
        [np.sin(rz),  np.cos(rz), 0, 0],
        [0,           0,          1, 0],
        [0,           0,          0, 1]
    ])
    Sh = np.array([
        [1,   0.4, 0, 0],
        [0, 1,   0, 0],
        [0, 0, 1,   0],
        [0,    0,    0,   1]
    ])
    R = Sh @ Rz @ Ry @ Rx
    T = apply_transform(R, cube)
    plot_3d(T, f"Rotation: x={np.degrees(rx):.0f}°, y={np.degrees(ry):.0f}°, z={np.degrees(rz):.0f}°")

interact(rotate_plot,
         rx=FloatSlider(value=0, min=-180, max=180, step=10),
         ry=FloatSlider(value=0, min=-180, max=180, step=10),
         rz=FloatSlider(value=0, min=-180, max=180, step=10))


In [ ]:

# === REFLECTION ===
def reflect_plot(reflect_xy=False, reflect_yz=False, reflect_xz=False):
    Rx = -1 if reflect_yz else 1
    Ry = -1 if reflect_xz else 1
    Rz = -1 if reflect_xy else 1

    Rf = np.array([
        [Rx, 0,  0, 0],
        [0,  Ry, 0, 0],
        [0,  0,  Rz, 0],
        [0,  0,  0, 1]
    ])
    T = apply_transform(Rf, cube)
    plot_3d(T, f"Reflection")

interact(reflect_plot,
        reflect_xy=Checkbox(value=False, description="Reflect about XY (Z flip)"),
        reflect_yz=Checkbox(value=False, description="Reflect about YZ (X flip)"),
        reflect_xz=Checkbox(value=False, description="Reflect about XZ (Y flip)"))